In [23]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
 
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
 
drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [33]:
%cd "/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier"

/content/drive/My Drive/Projects/quantum_image_classifier/PennyLane/Data Reuploading Classifier


In [1]:
!pip install pennylane
from IPython.display import clear_output
clear_output()

In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy as np

# Loading Raw Data

In [2]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train[:, 0:27, 0:27]
x_test = x_test[:, 0:27, 0:27]

In [3]:
x_train_flatten = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/255.0
x_test_flatten = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/255.0

In [4]:
print(x_train_flatten.shape, y_train.shape)
print(x_test_flatten.shape, y_test.shape)

(60000, 729) (60000,)
(10000, 729) (10000,)


In [5]:
x_train_0 = x_train_flatten[y_train == 0]
x_train_1 = x_train_flatten[y_train == 1]
x_train_2 = x_train_flatten[y_train == 2]
x_train_3 = x_train_flatten[y_train == 3]
x_train_4 = x_train_flatten[y_train == 4]
x_train_5 = x_train_flatten[y_train == 5]
x_train_6 = x_train_flatten[y_train == 6]
x_train_7 = x_train_flatten[y_train == 7]
x_train_8 = x_train_flatten[y_train == 8]
x_train_9 = x_train_flatten[y_train == 9]

x_train_list = [x_train_0, x_train_1, x_train_2, x_train_3, x_train_4, x_train_5, x_train_6, x_train_7, x_train_8, x_train_9]

print(x_train_0.shape)
print(x_train_1.shape)
print(x_train_2.shape)
print(x_train_3.shape)
print(x_train_4.shape)
print(x_train_5.shape)
print(x_train_6.shape)
print(x_train_7.shape)
print(x_train_8.shape)
print(x_train_9.shape)

(5923, 729)
(6742, 729)
(5958, 729)
(6131, 729)
(5842, 729)
(5421, 729)
(5918, 729)
(6265, 729)
(5851, 729)
(5949, 729)


In [6]:
x_test_0 = x_test_flatten[y_test == 0]
x_test_1 = x_test_flatten[y_test == 1]
x_test_2 = x_test_flatten[y_test == 2]
x_test_3 = x_test_flatten[y_test == 3]
x_test_4 = x_test_flatten[y_test == 4]
x_test_5 = x_test_flatten[y_test == 5]
x_test_6 = x_test_flatten[y_test == 6]
x_test_7 = x_test_flatten[y_test == 7]
x_test_8 = x_test_flatten[y_test == 8]
x_test_9 = x_test_flatten[y_test == 9]

x_test_list = [x_test_0, x_test_1, x_test_2, x_test_3, x_test_4, x_test_5, x_test_6, x_test_7, x_test_8, x_test_9]

print(x_test_0.shape)
print(x_test_1.shape)
print(x_test_2.shape)
print(x_test_3.shape)
print(x_test_4.shape)
print(x_test_5.shape)
print(x_test_6.shape)
print(x_test_7.shape)
print(x_test_8.shape)
print(x_test_9.shape)

(980, 729)
(1135, 729)
(1032, 729)
(1010, 729)
(982, 729)
(892, 729)
(958, 729)
(1028, 729)
(974, 729)
(1009, 729)


# Selecting the dataset

Output: X_train, Y_train, X_test, Y_test

In [7]:
X_train = np.concatenate((x_train_list[0][:200, :], x_train_list[1][:200, :]), axis=0)
Y_train = np.zeros((X_train.shape[0],), dtype=int)
Y_train[200:] += 1

X_train.shape, Y_train.shape

((400, 729), (400,))

In [8]:
X_test = np.concatenate((x_test_list[0][:500, :], x_test_list[1][:500, :]), axis=0)
Y_test = np.zeros((X_test.shape[0],), dtype=int)
Y_test[500:] += 1

X_test.shape, Y_test.shape

((1000, 729), (1000,))

# Dataset Preprocessing

In [9]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((400, 27, 27), (1000, 27, 27))

# Quantum

In [10]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

qml.enable_tape()

from tensorflow.keras.utils import to_categorical

# Set a random seed
np.random.seed(2020)

In [11]:
# Define output labels as quantum state vectors
def density_matrix(state):
    """Calculates the density matrix representation of a state.

    Args:
        state (array[complex]): array representing a quantum state vector

    Returns:
        dm: (array[complex]): array representing the density matrix
    """
    return state * np.conj(state).T


label_0 = [[1], [0]]
label_1 = [[0], [1]]
state_labels = [label_0, label_1]

In [12]:
n_qubits = 2  # number of class
dev_fc = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_fc)
def q_fc(params, inputs):
    """A variational quantum circuit representing the DRC.

    Args:
        params (array[float]): array of parameters
        inputs = [x, y]
        x (array[float]): 1-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]


In [13]:
dev_conv = qml.device("default.qubit", wires=9)


@qml.qnode(dev_conv)
def q_conv(conv_params, inputs):
    """A variational quantum circuit representing the Universal classifier + Conv.

    Args:
        params (array[float]): array of parameters
        x (array[float]): 2-d input vector
        y (array[float]): single output state density matrix

    Returns:
        float: fidelity between output state and input
    """
    # layer iteration
    for l in range(len(conv_params[0])):
        # RY layer
        # height iteration
        for i in range(3):
            # width iteration
            for j in range(3):
                qml.RY((conv_params[0][l][3*i+j] * inputs[i, j] + conv_params[1][l][3*i+j]), wires=(3*i+j))


    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3) @ qml.PauliZ(4) @ qml.PauliZ(5) @ qml.PauliZ(6) @ qml.PauliZ(7) @ qml.PauliZ(8))

In [14]:
a = np.zeros((2, 1, 9))
q_conv(a, X_train[0, 0:3, 0:3])

tensor(1., requires_grad=True)

In [15]:
a = np.zeros((2, 1, 9))
q_fc(a, X_train[0, 0, 0:9])

tensor([1., 0.], requires_grad=True)

In [16]:
from keras import backend as K


# Alpha Custom Layer
class class_weights(tf.keras.layers.Layer):
    def __init__(self):
        super(class_weights, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(1, 2), dtype="float32"),
            trainable=True,
        )

    def call(self, inputs):
        return (inputs * self.w)

In [24]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18*L*n_class + 2, output size = 2
num_fc_layer = 1
q_fc_layer_0 = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=2)(reshape_layer_3)

# Alpha Layer
alpha_layer_0 = class_weights()(q_fc_layer_0)

model = tf.keras.Model(inputs=X, outputs=alpha_layer_0)

In [25]:
model(X_train[0:3, :, :])

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.09844495, -0.00993194],
       [-0.09831073, -0.01002079],
       [-0.09858587, -0.00983866]], dtype=float32)>

In [26]:
import keras.backend as K

# def custom_loss(y_true, y_pred):
#     return K.sum(((y_true.shape[1]-2)*y_true+1)*K.square(y_true-y_pred))/len(y_true)

def custom_loss(y_true, y_pred):
    loss = K.square(y_true-y_pred)
    #class_weights = y_true*(weight_for_1-weight_for_0) + weight_for_0
    #loss = loss * class_weights
    
    return K.sum(loss)/len(y_true)

In [27]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss='mse', metrics=["accuracy"])

In [21]:
cp_val_acc = tf.keras.callbacks.ModelCheckpoint(filepath="./Model/2_QConv2noent_QFC_valacc.hdf5",
                monitor='val_accuracy', verbose=1, save_weights_only=True, save_best_only=True, mode='max')

cp_val_loss = tf.keras.callbacks.ModelCheckpoint(filepath="./Model/2_QConv2noent_QFC_valloss.hdf5",
                monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

In [22]:
filepath = "./Model_2/2_QConv2noent_QFC_saved-model-{epoch:02d}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                                                save_weights_only=True, save_best_only=False, mode='auto')

In [23]:
H = model.fit(X_train, to_categorical(Y_train), epochs=10, batch_size=32, initial_epoch=0,
              validation_data=(X_test, to_categorical(Y_test)), verbose=1,
              callbacks=[checkpoint, cp_val_acc, cp_val_loss])

Epoch 1/10
13/13 [==============================] - 5814s 459s/step - loss: 0.4210 - accuracy: 0.5390 - val_loss: 0.2447 - val_accuracy: 0.5000

Epoch 00001: saving model to ./Model_2/2_QConv2noent_QFC_saved-model-01.hdf5

Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to ./Model/2_QConv2noent_QFC_valacc.hdf5

Epoch 00001: val_loss improved from inf to 0.24468, saving model to ./Model/2_QConv2noent_QFC_valloss.hdf5
Epoch 2/10
13/13 [==============================] - 5887s 466s/step - loss: 0.2054 - accuracy: 0.6927 - val_loss: 0.0570 - val_accuracy: 0.9580

Epoch 00002: saving model to ./Model_2/2_QConv2noent_QFC_saved-model-02.hdf5

Epoch 00002: val_accuracy improved from 0.50000 to 0.95800, saving model to ./Model/2_QConv2noent_QFC_valacc.hdf5

Epoch 00002: val_loss improved from 0.24468 to 0.05704, saving model to ./Model/2_QConv2noent_QFC_valloss.hdf5
Epoch 3/10
13/13 [==============================] - 5921s 468s/step - loss: 0.0365 - accuracy: 0.9797 - val_l

In [39]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Layer (InputLayer)        [(None, 27, 27)]     0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_205 (S (None, 3, 3)         0           Input_Layer[0][0]                
__________________________________________________________________________________________________
tf.__operators__.getitem_206 (S (None, 3, 3)         0           Input_Layer[0][0]                
__________________________________________________________________________________________________
tf.__operators__.getitem_207 (S (None, 3, 3)         0           Input_Layer[0][0]                
____________________________________________________________________________________________

In [40]:
# RY noent
H.history

{'loss': [0.3190556466579437,
  0.1324336677789688,
  0.06383240222930908,
  0.04667748138308525,
  0.04163982719182968,
  0.034880828112363815,
  0.031560931354761124,
  0.03223029524087906,
  0.03223955258727074,
  0.029766449704766273],
 'accuracy': [0.5475000143051147,
  0.8774999976158142,
  0.9325000047683716,
  0.9524999856948853,
  0.9599999785423279,
  0.9624999761581421,
  0.9725000262260437,
  0.9649999737739563,
  0.9649999737739563,
  0.9725000262260437],
 'val_loss': [0.17143355309963226,
  0.11642319709062576,
  0.07859081029891968,
  0.05326811596751213,
  0.05914188176393509,
  0.05859171599149704,
  0.054212722927331924,
  0.05315721035003662,
  0.05396281182765961,
  0.05895267426967621],
 'val_accuracy': [0.8050000071525574,
  0.8489999771118164,
  0.9049999713897705,
  0.9480000138282776,
  0.9200000166893005,
  0.9210000038146973,
  0.9309999942779541,
  0.9350000023841858,
  0.9350000023841858,
  0.921999990940094]}

In [42]:
# RY noent
model.weights

[<tf.Variable 'model_1/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 2.8187332 , -0.47825494,  0.46355948, -0.50634617,
           0.00880399,  0.0429221 ,  0.01308273,  0.10211885,
          -0.12379515]],
 
        [[ 0.97048926, -0.31671068,  0.00362767, -0.09434931,
           0.03415625, -0.14301999,  0.01915532, -0.08899173,
           0.27037996]]], dtype=float32)>,
 <tf.Variable 'model_1/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.8531054 , -1.9238867 ,  0.5446696 ,  0.24465568,
           0.3157472 , -0.12321742, -0.22899951,  2.6124961 ,
           0.1721974 ]],
 
        [[-0.3874387 , -1.0143095 , -0.1570532 ,  0.19516507,
          -0.04742643,  0.00644725, -0.31880572, -1.2666101 ,
           0.08527368]]], dtype=float32)>,
 <tf.Variable 'model_1/keras_layer_1/params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 0.07617606, -2.2225084 ,  0.11496793,  1.3627197 ,
          -2.6583202 , 

# Exploring the results

In [44]:
X_train = np.concatenate((x_train_list[0][:20, :], x_train_list[1][:20, :]), axis=0)
Y_train = np.zeros((X_train.shape[0],), dtype=int)
Y_train[20:] += 1

X_train.shape, Y_train.shape

((40, 729), (40,))

In [45]:
X_test = np.concatenate((x_test_list[0][:20, :], x_test_list[1][:20, :]), axis=0)
Y_test = np.zeros((X_test.shape[0],), dtype=int)
Y_test[20:] += 1

X_test.shape, Y_test.shape

((40, 729), (40,))

In [46]:
X_train = X_train.reshape(X_train.shape[0], 27, 27)
X_test = X_test.reshape(X_test.shape[0], 27, 27)

X_train.shape, X_test.shape

((40, 27, 27), (40, 27, 27))

## First Layer

In [21]:
qconv_1_weights = np.array([[[ 2.8187332 , -0.47825494,  0.46355948, -0.50634617,
           0.00880399,  0.0429221 ,  0.01308273,  0.10211885,
          -0.12379515]],
 
        [[ 0.97048926, -0.31671068,  0.00362767, -0.09434931,
           0.03415625, -0.14301999,  0.01915532, -0.08899173,
           0.27037996]]])

qconv_1_weights.shape

(2, 1, 9)

In [49]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


qconv1_model = tf.keras.Model(inputs=X, outputs=reshape_layer_1)

In [50]:
qconv1_model(X_train[0:1])
qconv1_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])

In [51]:
qconv1_model.weights

[<tf.Variable 'model_2/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 2.8187332 , -0.47825494,  0.46355948, -0.50634617,
           0.00880399,  0.0429221 ,  0.01308273,  0.10211885,
          -0.12379515]],
 
        [[ 0.97048926, -0.31671068,  0.00362767, -0.09434931,
           0.03415625, -0.14301999,  0.01915532, -0.08899173,
           0.27037996]]], dtype=float32)>]

In [54]:
preprocessed_img_train = qconv1_model(X_train)

preprocessed_img_test = qconv1_model(X_test)

data_train = preprocessed_img_train.numpy().reshape(-1, 13*13)
np.savetxt('./2_QConv2noent_QFC-Filter1_Image_Train.txt', data_train)

data_test = preprocessed_img_test.numpy().reshape(-1, 13*13)
np.savetxt('./2_QConv2noent_QFC-Filter1_Image_Test.txt', data_test)

print(data_train.shape, data_test.shape)

(40, 169) (40, 169)


## Second Layer

In [22]:
qconv_2_weights = np.array([[[-0.8531054 , -1.9238867 ,  0.5446696 ,  0.24465568,
           0.3157472 , -0.12321742, -0.22899951,  2.6124961 ,
           0.1721974 ]],
 
        [[-0.3874387 , -1.0143095 , -0.1570532 ,  0.19516507,
          -0.04742643,  0.00644725, -0.31880572, -1.2666101 ,
           0.08527368]]])

qconv_2_weights.shape

(2, 1, 9)

In [56]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)



qconv2_model = tf.keras.Model(inputs=X, outputs=reshape_layer_2)

In [57]:
qconv2_model(X_train[0:1])
qconv2_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])
qconv2_model.get_layer('Quantum_Conv_Layer_2').set_weights([qconv_2_weights])

In [58]:
qconv2_model.weights

[<tf.Variable 'model_3/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[ 2.8187332 , -0.47825494,  0.46355948, -0.50634617,
           0.00880399,  0.0429221 ,  0.01308273,  0.10211885,
          -0.12379515]],
 
        [[ 0.97048926, -0.31671068,  0.00362767, -0.09434931,
           0.03415625, -0.14301999,  0.01915532, -0.08899173,
           0.27037996]]], dtype=float32)>,
 <tf.Variable 'model_3/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.8531054 , -1.9238867 ,  0.5446696 ,  0.24465568,
           0.3157472 , -0.12321742, -0.22899951,  2.6124961 ,
           0.1721974 ]],
 
        [[-0.3874387 , -1.0143095 , -0.1570532 ,  0.19516507,
          -0.04742643,  0.00644725, -0.31880572, -1.2666101 ,
           0.08527368]]], dtype=float32)>]

In [59]:
preprocessed_img_train = qconv2_model(X_train)

preprocessed_img_test = qconv2_model(X_test)

data_train = preprocessed_img_train.numpy().reshape(-1, 6*6)
np.savetxt('./2_QConv2noent_QFC-Filter2_Image_Train.txt', data_train)

data_test = preprocessed_img_test.numpy().reshape(-1, 6*6)
np.savetxt('./2_QConv2noent_QFC-Filter2_Image_Test.txt', data_test)

print(data_train.shape, data_test.shape)

(40, 36) (40, 36)


## Quantum States

In [23]:
X_train.shape, X_test.shape

((400, 27, 27), (1000, 27, 27))

In [24]:
q_fc_weights = np.array([[[ 0.07617606, -2.2225084 ,  0.11496793,  1.3627197 ,
          -2.6583202 , -0.28818664,  2.0106869 ,  0.20066635,
          -0.03082278]],
 
        [[ 0.20864335,  1.3935802 ,  0.48268482,  0.08141141,
           0.42544875, -0.02218557,  0.04886728,  0.6118612 ,
           0.37515417]]])

In [61]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [62]:
np.argmax(pred_train, axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], requires_grad=True)

In [63]:
np.argmax(pred_test, axis=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], requires_grad=True)

In [25]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)



maxpool_model = tf.keras.Model(inputs=X, outputs=reshape_layer_3)

In [26]:
maxpool_model(X_train[0:1])
maxpool_model.get_layer('Quantum_Conv_Layer_1').set_weights([qconv_1_weights])
maxpool_model.get_layer('Quantum_Conv_Layer_2').set_weights([qconv_2_weights])

In [27]:
maxpool_train = maxpool_model(X_train)
maxpool_test = maxpool_model(X_test)

maxpool_train.shape, maxpool_test.shape

(TensorShape([400, 9]), TensorShape([1000, 9]))

In [28]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [29]:
q_fc_state(np.zeros((2,1,9)), maxpool_train[0])

tensor(1., requires_grad=True)

In [30]:
q_fc_state(q_fc_weights, maxpool_train[0])

tensor(0.97627103, requires_grad=True)

In [31]:
train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

for i in range(len(train_state)):
    
    q_fc_state(q_fc_weights, maxpool_train[i])
    temp = np.flip(dev_state._state)
    train_state[i, :] = temp
    
for i in range(len(test_state)):
    
    q_fc_state(q_fc_weights, maxpool_test[i])
    temp = np.flip(dev_state._state)
    test_state[i, :] = temp

In [34]:
train_state.shape, test_state.shape

((400, 2), (1000, 2))

In [35]:
# sanity check

print(((np.conj(train_state) @ density_matrix(state_labels[0])) * train_state)[:, 0] > 0.5)

print(((np.conj(test_state) @ density_matrix(state_labels[0])) * test_state)[:, 0] > 0.5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False False  True  True  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [36]:
np.savetxt('./2_QConv2noent_QFC-State_Train_all_samples.txt', train_state)
np.savetxt('./2_QConv2noent_QFC-State_Test_all_samples.txt', test_state)

## Random Starting State

In [37]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)


# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(reshape_layer_1[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)


# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(reshape_layer_2)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


# Quantum FC Layer, trainable params = 18*L*n_class + 2, output size = 2
num_fc_layer = 1
q_fc_layer_0 = qml.qnn.KerasLayer(q_fc, {"params": (2, num_fc_layer, 9)}, output_dim=2)(reshape_layer_3)

# Alpha Layer
alpha_layer_0 = class_weights()(q_fc_layer_0)


model_random = tf.keras.Model(inputs=X, outputs=alpha_layer_0)
model_maxpool_random = tf.keras.Model(inputs=X, outputs=reshape_layer_3)

In [38]:
model_random(X_train[0:1])
model_random.weights

[<tf.Variable 'model_2/Quantum_Conv_Layer_1/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.15570569,  0.2688368 , -0.5205471 ,  0.5209966 ,
           0.43946862, -0.41040227, -0.01006722,  0.21328932,
           0.03991932]],
 
        [[ 0.3378569 , -0.30249137, -0.43277252, -0.33396918,
           0.19190145,  0.470232  ,  0.40253383,  0.5038419 ,
           0.29318976]]], dtype=float32)>,
 <tf.Variable 'model_2/Quantum_Conv_Layer_2/conv_params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.16110066, -0.287142  ,  0.44501978,  0.2118715 ,
           0.23768693, -0.17819047,  0.49069512, -0.43646657,
           0.18279302]],
 
        [[ 0.27290642,  0.08645177,  0.5218208 ,  0.3105942 ,
          -0.26338178, -0.19851625, -0.27393508, -0.5372756 ,
           0.30556488]]], dtype=float32)>,
 <tf.Variable 'model_2/keras_layer_1/params:0' shape=(2, 1, 9) dtype=float32, numpy=
 array([[[-0.35458952, -0.1612238 , -0.5396413 ,  0.1676842 ,
           0.34848082, 

In [39]:
random_weights = np.array([[[-0.35458952, -0.1612238 , -0.5396413 ,  0.1676842 ,
           0.34848082,  0.13427138, -0.22383437,  0.03579813,
          -0.01182914]],
 
        [[ 0.05027205, -0.16381726,  0.28698307, -0.1324831 ,
           0.33570415,  0.08612686,  0.24795538, -0.10930336,
          -0.447582  ]]])

In [40]:
maxpool_train = model_maxpool_random(X_train)
maxpool_test = model_maxpool_random(X_test)

maxpool_train.shape, maxpool_test.shape

(TensorShape([400, 9]), TensorShape([1000, 9]))

In [41]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [42]:
q_fc_state(np.zeros((2,1,9)), maxpool_train[0])

tensor(1., requires_grad=True)

In [43]:
q_fc_state(random_weights, maxpool_train[21])

tensor(0.9934085, requires_grad=True)

In [45]:
train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

for i in range(len(train_state)):
    
    q_fc_state(random_weights, maxpool_train[i])
    temp = np.flip(dev_state._state)
    train_state[i, :] = temp
    
for i in range(len(test_state)):
        
    q_fc_state(random_weights, maxpool_test[i])
    temp = np.flip(dev_state._state)
    test_state[i, :] = temp

In [47]:
train_state.shape, test_state.shape

((400, 2), (1000, 2))

In [56]:
# sanity check

print(((np.conj(train_state) @ density_matrix(state_labels[0])) * train_state)[:, 0] > 0.5)

print(((np.conj(test_state) @ density_matrix(state_labels[0])) * test_state)[:, 0] > 0.5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [58]:
np.savetxt('./2_QConv2noent_QFC-RandomState_Train_all_samples.txt', train_state)
np.savetxt('./2_QConv2noent_QFC-RandomState_Test_all_samples.txt', test_state)

# Finish

In [76]:
first_10_epoch = H.history
first_10_epoch

{'loss': [0.7249730825424194,
  0.485493928194046,
  0.2910504639148712,
  0.20750370621681213,
  0.1200886219739914,
  0.09333930909633636,
  0.09352166950702667,
  0.09991505742073059,
  0.0711403489112854,
  0.0675935298204422],
 'accuracy': [0.5249999761581421,
  0.6225000023841858,
  0.7825000286102295,
  0.8650000095367432,
  0.9449999928474426,
  0.9599999785423279,
  0.9624999761581421,
  0.9524999856948853,
  0.9750000238418579,
  0.9775000214576721],
 'val_loss': [0.6175757646560669,
  0.40504130721092224,
  0.3411727547645569,
  0.19497469067573547,
  0.11362368613481522,
  0.07933302223682404,
  0.1450268030166626,
  0.0960061326622963,
  0.09231595695018768,
  0.053687386214733124],
 'val_accuracy': [0.5,
  0.7300000190734863,
  0.7400000095367432,
  0.8700000047683716,
  0.9449999928474426,
  0.9800000190734863,
  0.925000011920929,
  0.9649999737739563,
  0.9549999833106995,
  0.9750000238418579]}

In [77]:
# initial 10 epoch
model.get_weights()

[array([[[ 0.13920915, -0.7933977 ,  0.4931736 , -0.34984022,
           0.41861725, -0.1258272 , -0.3323837 ,  1.4861933 ,
           0.14679849]],
 
        [[-0.48547882, -0.6672664 , -0.30831295,  0.31500554,
           0.21519628, -0.09552038, -0.33133912,  0.8978053 ,
          -0.38653338]]], dtype=float32),
 array([[[-0.04071528, -0.5614164 , -0.5472108 ,  0.32481748,
          -0.1586527 ,  0.06802678, -0.04312551, -1.0716503 ,
          -0.09168857]],
 
        [[ 0.2695058 , -1.5634912 ,  0.20198631, -0.20597708,
           0.06802317,  0.2522897 , -0.01191515,  0.5282674 ,
          -0.3303873 ]]], dtype=float32),
 array([[[-0.1229878 ,  0.03982117, -0.29502654, -1.2658846 ,
          -0.3401566 , -0.93464977,  1.1978221 , -0.24731389,
          -0.51143396]],
 
        [[-0.05537069, -1.3277841 , -0.7294856 , -0.740143  ,
          -0.83612764,  0.6652604 ,  0.0570471 , -0.4578703 ,
          -0.36179888]]], dtype=float32),
 array([[1.084761 , 1.0133266]], dtype=float32)]

# Making animations

In [36]:
H.history

{'loss': [0.3620220422744751,
  0.15828512609004974,
  0.031568512320518494,
  0.015363175421953201,
  0.012626182287931442,
  0.013658607378602028,
  0.010618376545608044,
  0.00844588316977024,
  0.00741078844293952,
  0.00828629732131958],
 'accuracy': [0.5350000262260437,
  0.8025000095367432,
  0.9775000214576721,
  0.987500011920929,
  0.9900000095367432,
  0.987500011920929,
  0.9925000071525574,
  0.9950000047683716,
  0.9950000047683716,
  0.9950000047683716],
 'val_loss': [0.24467992782592773,
  0.05703796073794365,
  0.02196177840232849,
  0.013408777303993702,
  0.009689709171652794,
  0.009040659293532372,
  0.009525229223072529,
  0.005966325290501118,
  0.007606671191751957,
  0.006261697970330715],
 'val_accuracy': [0.5,
  0.9580000042915344,
  0.9800000190734863,
  0.9829999804496765,
  0.9950000047683716,
  0.9940000176429749,
  0.9959999918937683,
  0.996999979019165,
  0.9940000176429749,
  0.9959999918937683]}

In [28]:
random_weights = model.get_weights()
random_weights

[array([[[ 0.15955305, -0.36515248, -0.20135769, -0.5028946 ,
           0.03527224, -0.5343016 , -0.49984446, -0.22550562,
          -0.41205758]],
 
        [[-0.08831877, -0.4926982 ,  0.3360647 , -0.18290153,
           0.20178366,  0.43514174, -0.14799583, -0.09449527,
          -0.3789652 ]]], dtype=float32),
 array([[[-0.35645655,  0.3122632 , -0.20967597, -0.05964521,
          -0.02155304, -0.2847187 , -0.2588721 , -0.12330633,
          -0.02357125]],
 
        [[-0.51691824, -0.02189517, -0.40646034, -0.08631384,
           0.10867143, -0.1307399 , -0.34200495,  0.2594096 ,
          -0.42972147]]], dtype=float32),
 array([[[ 0.05261153, -0.09614277,  0.43770754,  0.2689674 ,
          -0.11526647,  0.3181463 , -0.1778247 , -0.31703696,
           0.333638  ]],
 
        [[-0.13681078,  0.5402223 ,  0.45211053,  0.468462  ,
           0.02845639, -0.38620314,  0.35767323, -0.52100194,
          -0.44384956]]], dtype=float32),
 array([[-0.11344912, -0.07509711]], dtype=float3

In [37]:
weights_list = [random_weights]

for i in range(10):
    if i == 9:
        model.load_weights('./Model_2/2_QConv2noent_QFC_saved-model-' + str(i+1) + '.hdf5')
        weights_list += [model.get_weights()]
    else:
        model.load_weights('./Model_2/2_QConv2noent_QFC_saved-model-0' + str(i+1) + '.hdf5')
        weights_list += [model.get_weights()]
        
    
len(weights_list)

11

In [38]:
# Input image, size = 27 x 27
X = tf.keras.Input(shape=(27,27), name='Input_Layer')


# Specs for Conv
c_filter = 3
c_strides = 2


# First Quantum Conv Layer, trainable params = 18*L, output size = 13 x 13
num_conv_layer_1 = 1
q_conv_layer_1 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_1, 9)}, output_dim=(1), name='Quantum_Conv_Layer_1')
size_1 = int(1+(X.shape[1]-c_filter)/c_strides)
q_conv_layer_1_list = []
# height iteration
for i in range(size_1):
    # width iteration
    for j in range(size_1):
        temp = q_conv_layer_1(X[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_1_list += [temp]
concat_layer_1 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_1_list)
reshape_layer_1 = tf.keras.layers.Reshape((size_1, size_1))(concat_layer_1)

qconv1_model = tf.keras.Model(inputs=X, outputs=reshape_layer_1)

In [39]:
X_2 = tf.keras.Input(shape=(size_1,size_1), name='Input_Layer_2')

# Second Quantum Conv Layer, trainable params = 18*L, output size = 6 x 6
num_conv_layer_2 = 1
q_conv_layer_2 = qml.qnn.KerasLayer(q_conv, {"conv_params": (2, num_conv_layer_2, 9)}, output_dim=(1), name='Quantum_Conv_Layer_2')
size_2 = int(1+(reshape_layer_1.shape[1]-c_filter)/c_strides)
q_conv_layer_2_list = []
# height iteration
for i in range(size_2):
    # width iteration
    for j in range(size_2):
        temp = q_conv_layer_2(X_2[:, 2*i:2*(i+1)+1, 2*j:2*(j+1)+1])
        temp = tf.keras.layers.Reshape((1,))(temp)
        q_conv_layer_2_list += [temp]
concat_layer_2 = tf.keras.layers.Concatenate(axis=1)(q_conv_layer_2_list)
reshape_layer_2 = tf.keras.layers.Reshape((size_2, size_2, 1))(concat_layer_2)

qconv2_model = tf.keras.Model(inputs=X_2, outputs=reshape_layer_2)

In [40]:
X_3 = tf.keras.Input(shape=(size_2,size_2,1), name='Input_Layer_3')

# Max Pooling Layer, output size = 9
max_pool_layer = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name='Max_Pool_Layer')(X_3)
reshape_layer_3 = tf.keras.layers.Reshape((9,))(max_pool_layer)


maxpool_model = tf.keras.Model(inputs=X_3, outputs=reshape_layer_3)

In [41]:
n_qubits = 1  # number of class
dev_state = qml.device("default.qubit", wires=n_qubits)


@qml.qnode(dev_state)
def q_fc_state(params, inputs):
    
    # layer iteration
    for l in range(len(params[0])):
        # qubit iteration
        for q in range(n_qubits):
            # gate iteration
            for g in range(int(len(inputs)/3)):
                qml.Rot(*(params[0][l][3*g:3*(g+1)] * inputs[3*g:3*(g+1)] + params[1][l][3*g:3*(g+1)]), wires=q)
    
    #return [qml.expval(qml.Hermitian(density_matrix(state_labels[i]), wires=[i])) for i in range(n_qubits)]
    return qml.expval(qml.Hermitian(density_matrix(state_labels[0]), wires=[0]))

In [42]:
a = qconv1_model(X_train[0:1])
b = qconv2_model(a)
c = maxpool_model(b)
d = q_fc_state(weights_list[0][2], c)
print(d)
print('ok')

1.0
ok


In [43]:
X_train.shape, X_test.shape

((400, 27, 27), (1000, 27, 27))

In [44]:
for k in range(11):

    qconv1_model.get_layer('Quantum_Conv_Layer_1').set_weights([weights_list[k][0]])
    qconv2_model.get_layer('Quantum_Conv_Layer_2').set_weights([weights_list[k][1]])
    print('Model ' + str(k) + ' got loaded.')


    preprocessed_img_train_1 = qconv1_model(X_train)
    preprocessed_img_test_1 = qconv1_model(X_test)

    data_train = preprocessed_img_train_1.numpy().reshape(-1, 13*13)
    np.savetxt('./Model_2/2_QConv2noent_QFC/2_QConv2noent_QFC-Filter1_Image_Train_Epoch=' + str(k) + '.txt', data_train)
    data_test = preprocessed_img_test_1.numpy().reshape(-1, 13*13)
    np.savetxt('./Model_2/2_QConv2noent_QFC/2_QConv2noent_QFC-Filter1_Image_Test_Epoch=' + str(k) + '.txt', data_test)
    print('Filter 1 finished.')


    preprocessed_img_train_2 = qconv2_model(preprocessed_img_train_1)
    preprocessed_img_test_2 = qconv2_model(preprocessed_img_test_1)

    data_train = preprocessed_img_train_2.numpy().reshape(-1, 6*6)
    np.savetxt('./Model_2/2_QConv2noent_QFC/2_QConv2noent_QFC-Filter2_Image_Train_Epoch=' + str(k) + '.txt', data_train)
    data_test = preprocessed_img_test_2.numpy().reshape(-1, 6*6)
    np.savetxt('./Model_2/2_QConv2noent_QFC/2_QConv2noent_QFC-Filter2_Image_Test_Epoch=' + str(k) + '.txt', data_test)
    print('Filter 2 finished.')


    maxpool_train = maxpool_model(preprocessed_img_train_2)
    maxpool_test = maxpool_model(preprocessed_img_test_2)

    train_state = np.zeros((len(X_train), 2), dtype=np.complex_)
    test_state = np.zeros((len(X_test), 2), dtype=np.complex_)

    for i in range(len(train_state)):

        q_fc_state(weights_list[k][2], maxpool_train[i])
        temp = np.flip(dev_state._state)
        train_state[i, :] = temp

    for i in range(len(test_state)):
        q_fc_state(weights_list[k][2], maxpool_test[i])
        temp = np.flip(dev_state._state)
        test_state[i, :] = temp

    np.savetxt('./Model_2/2_QConv2noent_QFC/2_QConv2noent_QFC-State_Train_Epoch=' + str(k) + '.txt', train_state)
    np.savetxt('./Model_2/2_QConv2noent_QFC/2_QConv2noent_QFC-State_Test_Epoch=' + str(k) + '.txt', test_state)
    print('Quantum State finished.')

Model 0 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 1 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 2 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 3 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 4 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 5 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 6 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 7 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 8 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 9 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
Model 10 got loaded.
Filter 1 finished.
Filter 2 finished.
Quantum State finished.
